In [1]:
from google.colab import drive
import sys
import os

drive.mount('/content/drive')

# Personalized path to your project directory
path = '/content/drive/MyDrive/cod/LEA3_health_7' # Replace with your actual path

# Add the path to sys.path
sys.path.append(path)
os.chdir(path)

os.curdir


Mounted at /content/drive


'.'

In [2]:
import numpy as np
import joblib ### para cargar array

########Paquetes para NN #########

from sklearn import metrics ### para analizar modelo
from sklearn.ensemble import RandomForestClassifier  ### para analizar modelo
import pandas as pd

from sklearn import tree
import tensorflow as tf


import cv2 ### para leer imagenes jpeg
### pip install opencv-python

from matplotlib import pyplot as plt #

from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
x_train = joblib.load('salidas/x_train.pkl')
y_train = joblib.load('salidas/y_train.pkl')
x_test = joblib.load('salidas/x_test.pkl')
y_test = joblib.load('salidas/y_test.pkl')

Si hay pocas imágenes de cáncer (melanoma, carcinoma) comparadas con benignas hay un problema desbalanceado.

¿Qué es más grave? ¿Decir que alguien no tiene cáncer cuando sí tiene? (falso negativo) f1-score o recall

¿O decir que alguien tiene cáncer cuando no es? (falso positivo)

In [8]:

################ Modelos Shallow learning ##############
##############################################################################

x_train.shape
y_train.shape


(8012, 37517)

In [13]:
##### validamos que sea un archivo de 1d ############
x_train[1]

array([159, 142, 158, ..., False, False, False], dtype=object)

In [ ]:

############### Indicador ################


### Recall: Tasa de identficación de positivos (de los que tenían neumonía cuantos predije tenían)
### F1_Socre: Combina Precions y recall (promedio entre dos anteriores)

### AUC: detección de positivos vs mala clasificaicón de negativos: porcentaje de los que neumonía que identifico vs los normales que digo que tiene neumonía

############Analisis problema ###########
#### me interesa recall: queremos detectar la mayor cantidad posible de casos reales de cáncer (minimizar falsos negativos). No queremos que un cáncer pase desapercibido.
#### el problema es que puede generar mucho trabajo porque clasifica a
####la mayoria como con no cancer (revisar), entonces usaremos el AUC que mide la capacidad e clasificación de cancer en balance con los que no tiene cancer, mal calsificados


In [ ]:
#################### RandomForest ##########

# Definir etiquetas para continuar

rf=RandomForestClassifier()
rf.fit(x_train, y_train)

pred_train=rf.predict(x_train)
print(metrics.classification_report(y_train, pred_train))


pred_test=rf.predict(x_test)
print(metrics.classification_report(y_test, pred_test))

print(metrics.roc_auc_score(y_train, pred_train))
print(metrics.roc_auc_score(y_test, pred_test))

cm=metrics.confusion_matrix(y_test,pred_test, labels=[1,0])
disp=metrics.ConfusionMatrixDisplay(cm,display_labels=['No_Cancer', 'Cancer']) ## (Cambiar etiquetas)
disp.plot()

In [ ]:
########### Decsion tree ##########

clf_dt= tree.DecisionTreeClassifier()
clf=clf_dt.fit(x_train, y_train)

pred_train=clf_dt.predict(x_train)
print(metrics.classification_report(y_train, pred_train))
metrics.roc_auc_score(y_train, pred_train)

pred_test=clf_dt.predict(x_test)
print(metrics.classification_report(y_test, pred_test))
metrics.roc_auc_score(y_test, pred_test)


In [ ]:

######## Redes neuronales convencionales #########


fc_model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=x_train.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


checkpoint = ModelCheckpoint(
    filepath='salidas/best_model_auc.keras',
    monitor='val_recall',        # Validar la metrica que se va a escoger de momento Recall
    mode='max',               # Usar max ya que buscamos maximizar la metrica en este punto debido a que representa un mejor resultado
    save_best_only=True,
    verbose=1
)

##### configura el optimizador y la función para optimizar ##############

fc_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','AUC', 'Recall', 'Precision'])


###### Entrenar el modelo usando el optimizador y arquitectura definidas #########
fc_model.fit(x_train, y_train, batch_size=100, epochs=10, validation_data=(x_test, y_test), callbacks=checkpoint)


######### Evaluar el modelo ####################
test_loss, test_acc, test_auc, test_recall, test_precision = fc_model.evaluate(x_test, y_test, verbose=2)
print("Test auc:", test_auc)

In [ ]:
best_auc=tf.keras.models.load_model('salidas/best_model_auc.keras')
test_loss, test_acc, test_auc, test_recall, test_precision = best_auc.evaluate(x_test, y_test, verbose=2)
print("Test auc:", test_auc)


In [ ]:
###### matriz de confusión test
pred_test=(best_auc.predict(x_test) > 0.9).astype('int')
cm=metrics.confusion_matrix(y_test,pred_test, labels=[1,0])
disp=metrics.ConfusionMatrixDisplay(cm,display_labels=['No_Cancer', 'Cancer']) # Validar cambiod e etiquetas
disp.plot()

print(metrics.classification_report(y_test, pred_test))


In [ ]:
## Validar codigo
# Predicción con umbral personalizado
pred_test = (best_auc.predict(x_test) > 0.9).astype('int')

# Etiquetas: 1 = Cancer (positivo), 0 = No_Cancer (negativo)
cm = metrics.confusion_matrix(y_test, pred_test, labels=[1, 0])

# Mostrar la matriz con etiquetas correctas
disp = metrics.ConfusionMatrixDisplay (
    confusion_matrix=cm,
    display_labels=['Cancer', 'No_Cancer']) # validar cambios en etiquetas
disp.plot(cmap='Blues')  # mejor visualización
plt.title('Matriz de Confusión - Test')
plt.show()

# Reporte de métricas por clase
print(metrics.classification_report(y_test, pred_test, target_names=['No_Cancer', 'Cancer']))
